In [ ]:
from typing import Tuple, Literal, TypeVar

from numpy import integer, bool_, floating, ndarray, s_, ones
from plotly.express import line

from sample_degradation.processing_stages import (
    load_data,
    mask_frames,
    estimate_poisson_uncertainty,
    correct_deadtime,
    correct_dark_current,
    normalize_frame_time,
    normalize_transmitted_flux,
    correct_self_absorbtion,
    average_frames,
    subtract_background,
    correct_flatfield,
    correct_angular_efficincy,
    correct_solid_angle,
    correct_polarization,
    normalize_thickness,
    azimuthally_integrate
)
from sample_degradation.utils.hdf5_filters import load_hdf5_filters
from sample_degradation.utils.visualization import show_frame
from sample_degradation.utils.get_background import get_background


In [ ]:
FILE_PATH = "/dls/i22/data/2021/sm27656-2/i22-588299.nxs"
BACKGROUND_PATH = get_background(FILE_PATH)

ABSORBTION_COEFF = 0.01

In [ ]:
load_hdf5_filters()
NumFrames = TypeVar("NumFrames", bound=int)
NumBackgrounds = TypeVar("NumBackgrounds", bound=int)
frames: ndarray[Tuple[NumFrames, Literal[1679], Literal[1475]], integer] = load_data(FILE_PATH, "entry1/instrument/detector/data", s_[740:750, 0, :, :]).squeeze()
background: ndarray[Tuple[NumBackgrounds, Literal[1679], Literal[1475]], integer] = load_data(BACKGROUND_PATH, "entry1/instrument/detector/data", s_[0, :10, :, :]).squeeze()

In [ ]:
mask: ndarray[Tuple[Literal[1679], Literal[1475]], bool_] = load_data(FILE_PATH, "entry1/instrument/detector/pixel_mask")
frames = mask_frames(frames, mask)
background = mask_frames(background, mask)

In [ ]:
uncert_frames = estimate_poisson_uncertainty(frames)
uncert_background = estimate_poisson_uncertainty(background)

In [ ]:
count_times: ndarray[Tuple[Literal[10]], floating] = load_data(FILE_PATH, "entry1/instrument/detector/count_time", s_[740:750, 0]).squeeze()

uncert_frames = correct_deadtime(uncert_frames, count_times, 3E-6, 2E-6)
uncert_background = correct_deadtime(uncert_background, count_times[0], 3E-6, 2E-6)

In [ ]:
uncert_frames = correct_dark_current(uncert_frames, count_times, 0.0, 0.0, 0.0)
uncert_background = correct_dark_current(uncert_background, count_times, 0.0, 0.0, 0.0)

In [ ]:
uncert_frames = normalize_frame_time(uncert_frames, count_times)
uncert_background = normalize_frame_time(uncert_background, count_times[0])

In [ ]:
uncert_frames = normalize_transmitted_flux(uncert_frames)
uncert_background = normalize_transmitted_flux(uncert_background)

In [ ]:
beam_center_pixels = (
    load_data(FILE_PATH, "entry1/instrument/detector/beam_center_y"),
    load_data(FILE_PATH, "entry1/instrument/detector/beam_center_x")
)
pixel_sizes = (
    load_data(FILE_PATH, "entry1/instrument/detector/y_pixel_size"),
    load_data(FILE_PATH, "entry1/instrument/detector/x_pixel_size")
)
distance = load_data(FILE_PATH, "entry1/instrument/detector/distance")
thickness = load_data(FILE_PATH, "entry1/sample/thickness")


uncert_frames = correct_self_absorbtion(
    uncert_frames,
    beam_center_pixels,
    pixel_sizes,
    distance,
    ABSORBTION_COEFF,
    thickness
    )
uncert_background = correct_self_absorbtion(
    uncert_background,
    beam_center_pixels,
    pixel_sizes,
    distance,
    ABSORBTION_COEFF,
    thickness
)

In [ ]:
azimuthally_intensities, azimuths = azimuthally_integrate(
    uncert_frames[0]["nominal"],
    beam_center_pixels,
    pixel_sizes,
    distance
)
line(x=azimuths, y=azimuthally_intensities, log_y=True)

In [ ]:
uncert_background = average_frames(uncert_background)

In [ ]:
diff_frames = subtract_background(uncert_frames, uncert_background)

In [ ]:
diff_frames = correct_flatfield(diff_frames, ones(diff_frames.shape[1:], dtype=floating))

In [ ]:
diff_frames = correct_angular_efficincy(
    diff_frames,
    beam_center_pixels,
    pixel_sizes,
    distance,
    ABSORBTION_COEFF,
    thickness
)

In [ ]:
diff_frames = correct_solid_angle(
    diff_frames,
    beam_center_pixels,
    pixel_sizes,
    distance
)

In [ ]:
diff_frames = correct_polarization(
    diff_frames,
    beam_center_pixels,
    pixel_sizes,
    distance,
    0.99
)

In [ ]:
diff_frames = normalize_thickness(
    diff_frames,
    1E-3
)

In [ ]:
show_frame(diff_frames[0]["nominal"].filled(0.0))